In [1]:
import numpy as np
import pandas as pd
import folium
import geocoder
from requests import Session

In [2]:
# numbeo = pd.read_csv('data/numbeo.csv')

In [2]:
# Load the cleaned CSV from the cell above
# Only run the subsequent cells if you want to gather fresh data
numbeo_beer = pd.read_table('data/numbeo_beer.txt', names=["rank", "city", "beer"], index_col="rank")
numbeo_bread = pd.read_table('data/numbeo_bread.txt', names=["rank", "city", "bread"], index_col="rank")
numbeo_coffee = pd.read_table("data/numbeo_coffee.txt", names=['rank', 'city', 'coffee'], index_col="rank")
numbeo = pd.merge(numbeo_beer, numbeo_bread, how="outer", on="city")
numbeo = pd.merge(numbeo, numbeo_coffee, how="outer", on="city")

In [3]:
split = numbeo['city'].str.split(", ")
numbeo['country'] = split.str[-1]
numbeo['city'] = split.str[0]

In [4]:
city_data = pd.read_csv('data/city-data/world-cities-data.csv')

In [5]:
numbeo = numbeo.rename(columns={"city": "city_ascii"})
numbeo = pd.merge(numbeo, city_data[["city_ascii", 'country','region', 'latitude', 'longitude', 'population', "admin_name"]], how='left', on=['city_ascii', 'country'])

,city_ascii,beer,bread,coffee,country,region,latitude,longitude,population,admin_name
0,Doha,8.51,1.45,4.63,Qatar,Middle east,25.3000,51.5333,1312947.0,Ad Dawḩah
1,Amman,5.30,0.41,4.14,Jordan,Middle east,31.9500,35.9333,4007526.0,Al ‘Āşimah
2,Newcastle,4.70,1.67,3.52,Australia,Asia & Pacific,-32.9167,151.7500,322278.0,New South Wales
3,Muscat,4.66,1.08,4.61,Oman,Middle east,23.6139,58.5922,1421409.0,Masqaţ
4,Melbourne,4.57,2.34,3.40,Australia,Asia & Pacific,-37.8136,144.9631,5078193.0,Victoria


In [6]:
numbeo.head(25)

,city_ascii,beer,bread,coffee,country,region,latitude,longitude,population,admin_name
0,Doha,8.51,1.45,4.63,Qatar,Middle east,25.3000,51.5333,1312947.0,Ad Dawḩah
1,Amman,5.30,0.41,4.14,Jordan,Middle east,31.9500,35.9333,4007526.0,Al ‘Āşimah
2,Newcastle,4.70,1.67,3.52,Australia,Asia & Pacific,-32.9167,151.7500,322278.0,New South Wales
3,Muscat,4.66,1.08,4.61,Oman,Middle east,23.6139,58.5922,1421409.0,Masqaţ
4,Melbourne,4.57,2.34,3.40,Australia,Asia & Pacific,-37.8136,144.9631,5078193.0,Victoria
5,Adelaide,4.54,2.10,3.31,Australia,Asia & Pacific,-34.9289,138.6011,1345777.0,South Australia
6,Trondheim,4.52,3.66,5.21,Norway,Europe,63.4400,10.4000,183378.0,Sør-Trøndelag
7,Brisbane,4.50,2.12,3.55,Australia,Asia & Pacific,-27.4678,153.0281,2514184.0,Queensland
8,Canberra,4.26,2.57,3.45,Australia,Asia & Pacific,-35.2931,149.1269,426704.0,Australian Capital Territory
9,Sydney,4.09,2.05,3.27,Australia,Asia & Pacific,-33.8650,151.2094,5312163.0,New South Wales


In [6]:
numbeo.to_csv("data/numbeo.csv")

In [7]:
import qwikidata.sparql

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out

In [8]:
# This cell takes >10min to run D:
import time

numbeo['population2'] = np.zeros(len(numbeo), dtype=int)
for city, country in zip(numbeo["city_ascii"], numbeo['country']):
    try:
        data = get_city_wikidata(city, country)
        numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), "population2"] = int(data['population']['value'])
    except:
        numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), "population2"] = np.nan
    time.sleep(1)

In [9]:
# This cell also takes a while
numbeo["latitude2"] = np.zeros(len(numbeo))
numbeo['longitude2'] = np.zeros(len(numbeo))
with Session() as s:
    for city, country in zip(numbeo["city_ascii"], numbeo['country']):
        latlng = geocoder.osm(city + ", " + country, session=s).latlng
        if latlng:
            numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), 'latitude2'] = latlng[0]
            numbeo.loc[(numbeo['city_ascii'] == city) & (numbeo['country'] == country), 'longitude2'] = latlng[1]
        time.sleep(1)

In [10]:
# For some reason, one city consistently returns None during the for loop
# while None in locations:
#     i = locations.index(None)
#     locations[i] = geocoder.osm(numbeo["city"][i]).latlng

In [26]:
# Start running the notebook again here
m = folium.Map(location=(numbeo["latitude"][1], numbeo["longitude"][1]), zoom_start=1)
for coord, price in zip(zip(numbeo['latitude'], numbeo['longitude']), numbeo['beer']):
    folium.Marker(location=coord, popup=price, tooltip=price).add_to(m)

ValueError: Location values cannot contain NaNs.

In [17]:
m

In [23]:
list(zip(zip([1, 2, 3], [4, 5, 6]), [7, 8, 9]))

[((1, 4), 7), ((2, 5), 8), ((3, 6), 9)]